# NIST Web Scraping

The following code documents the methods of combining and organizing the scraped data.

## Scraping code

The web scraping was conducted through the following code. Increments of 27 reference ID URLs were scraped to maintain accuracy of the scraper while executing a timely web scrape.

The above scraping protocols were refined and consolidated. The final dataframe consists of the desired columns from all components of the NIST website and the static HTML file. A 30-second time delay was introduced to the end of each scraping loop, which theoretically allows the NIST server to recover and critically improves the accuracy of the scraped values to an acceptable threshold. 

In [ ]:
# Import Statements
from bs4 import BeautifulSoup
import requests #Pulling webpages
import pandas
import datetime
import numpy
import os
import re
import time
import math

# defining the website
root_url = "https://randr.nist.gov/enzyme/DataDetails.aspx?ID="
end_url = "&finalterm=&data=enzyme"
#===========================================================================================================================

# identify the table and rows of pertinent data
bs = BeautifulSoup(open('Enzyme Thermodynamic Database.html'), 'lxml')
table = bs.find("table", attrs = {'id': 'MainBody_gvSearch'})
body = table.find_all("tr")
#print(body[1])

# defining the boundaries of the dataframe section
index_range = 12000
index_count = 0
loop_count = 0 
output_loop = 1
lower_bound = 1 #math.floor(1*len(body)/50)
upper_bound = math.floor(1*len(body))

# loop through the enzyme id values 
name_iteration = 0
enzyme_iteration = 0
for row in range(lower_bound,upper_bound):   
    # parsing the reaction names and strings   
    enzyme_name = body[row].find('span', attrs = {'id': 'MainBody_gvSearch_lblEnzyme_%s' %(name_iteration)}).text
    reaction = body[row].find('span', attrs = {'id': 'MainBody_gvSearch_lblReaction_%s' %(name_iteration)}).text
    id_value = body[row].find("a").text
    name_iteration += 1
    
    # defining the soup
    total_url = root_url + id_value + end_url
    soup = requests.get(total_url).text
    bs = BeautifulSoup(soup, 'lxml')
    
    # scrape the table and header information
    tables1 = bs.find_all("table", attrs = {"id": "MainBody_extraData"})
    print(id_value, '\t: ', loop_count, '\t, ', len(tables1))
    if len(tables1) != 1:
        continue

    body1 = tables1[0].find_all("tr")
    body_rows1 = body1[1:]
    heads = body1[0]

    headings = ['Enzyme:', 'Reaction:']
    for head in heads.find_all("th"):
        head = (head.text).rstrip("\n")
        headings.append(head)
        
    #print(headings)

    total_rows = []
    for row_number in range(len(body_rows1)):
        each_row = [enzyme_name, reaction]
        for row_element in body_rows1[row_number].find_all("td"):
            row_refined = re.sub("(\xa0)|(\n)|,","",row_element.text)
            each_row.append(row_refined)
        total_rows.append(each_row)
        
    #print(total_rows)
    
    # create a dataframe
    index_list_body = range(index_count+lower_bound-1, len(body_rows1)+index_count+lower_bound-1)
    bs_dataframe_table1 = pandas.DataFrame(data = total_rows, columns = headings, index = index_list_body)
    bs_dataframe_table1.drop(bs_dataframe_table1.columns[len(bs_dataframe_table1.columns)-1], axis=1, inplace=True)
    '''display(bs_dataframe_table1)'''
    
#===========================================================================================================================

    # scrape additional information 
    tables = bs.find_all("table", attrs={"id": "MainBody_DataList1"})
    if len(tables) == 0:
        continue
        print('Failed reference ID: ', id_value)
        
    body2 = tables[0].find_all("tr")
    body_rows2 = body2[1:]

    each_row2 = []
    for row in range(len(body_rows2)):
        for row_element in body_rows2[row].find_all("td"):
            #print('row element: ', row_element)
            row_refined2 = re.sub("(\xa0)|(\n)|,","",row_element.text)
            #print('row refined', row_refined2)
            each_row2.append(row_refined2)

    information_entries_list = []
    information_values_list = []
    column_count = 0
    for i, element in enumerate(each_row2):
        if i == 0 or i % 2 == 0:
            information_entries_list.append(element)
            column_count += 1
        else:
            information_values_list.append(element)
            column_count += 1
    column_count /= 2

    # create the dataframe and refine the columns
    index_list_reference = range(index_count+lower_bound-1, index_count+1+lower_bound-1)
    bs_dataframe_pretable2 = pandas.DataFrame(data = [information_values_list], columns = information_entries_list, index = index_list_reference)
    bs_dataframe_pretable2.drop(bs_dataframe_pretable2.columns[len(bs_dataframe_pretable2.columns)-2], axis=1, inplace=True)
    bs_dataframe_pretable2.drop(bs_dataframe_pretable2.columns[len(bs_dataframe_pretable2.columns)-1], axis=1, inplace=True)
    '''display(bs_dataframe_pretable2)'''
    
#===========================================================================================================================

    # merge the dataframes of this loop
    this_dataframe = bs_dataframe_table1.join(bs_dataframe_pretable2)
    this_dataframe.index.name = 'index'
    
    # iteratively coalesce the new dataframe into the old dataframe 
    if loop_count == 0:
        old_dataframe = this_dataframe
        old_dataframe.index.name = 'index'
        
    elif loop_count > 0:
        these_columns = []
        for column in this_dataframe:
            these_columns.append(column)
            
        old_columns = []
        for existing_column in old_dataframe:
            old_columns.append(existing_column)
            
        common_columns = list(set(these_columns).intersection(old_columns))
        
        # amalgamate the dataframe with the pre-existing dataframe
        current_dataframe = old_dataframe.merge(this_dataframe, on = common_columns, how = 'outer')
        old_dataframe = current_dataframe  
        
    # amalgamate the dataframe with the pre-existing dataframe
    index_count += len(body_rows1)
    
    time_delay = 0
    time.sleep(time_delay)
    
    max_referenes_per_csv = 0
    if loop_count == max_referenes_per_csv:
        id_value = re.sub('(/)', '-', id_value)
        output = './individual scraping/{}, {}.csv'.format(datetime.date.today(), id_value, output_loop)
        while os.path.exists(output):
            output_loop += 1
            output = './individual scraping/2021-05-06_{}.csv'.format(datetime.date.today(), id_value, output_loop)
            
        old_dataframe.to_csv(output)
        
        loop_count = 0 
        
    else:        
        loop_count += 1 

## CSV data combination

The following code was combined the individual CSV files. The code was dynamically applied to aggregate CSV files into the segments 1-5, 6-16, 17-36, and 37-50. The code subsequently combined each of the four segments into the complete csv file. The columns were reorganized according to subjective preference with the reference information on the left and the data on the right.

Efficiently combining the folder of individually scraped CSV files from the NIST database. The columns are reorganized, and the final CSV is exported.

In [ ]:
# import libraries
import pandas
import numpy
import glob
import os
import re

#path_original = "./sabio_scraped/"
path = './individual scraping/'
files = glob.glob(os.path.join(path, '*.csv'))

# create the total list of dataframes
'''print(len(files))'''
total_dataframes = []
for file in files:
    '''re.sub('(\\\)', '(/)', file)
    print(file)'''
    dfn = pandas.read_csv(file)
    total_dataframes.append(dfn)
    
# combine the total set of dataframes
combined_df = pandas.DataFrame()
combined_df = pandas.concat(total_dataframes)
display(combined_df)

# refine the dataframe 
combined_df = combined_df.fillna(' ')
middle_dataframe_columns = ['T(K)', 'pH ', 'K<sub>c</sub>\' ', 'δ<sub>r</sub>H\'<sup>o</sup>(kJ.mol<sup>-1</sup>)', 'Km\'']
left_dataframe_columns = ['index', 'Enzyme:', 'Reaction:', 'Reference:', 'Reference ID:'] 
right_dataframe_columns = list(set(combined_df.columns) - set(left_dataframe_columns) - set(middle_dataframe_columns))
defined_columns = left_dataframe_columns + middle_dataframe_columns + right_dataframe_columns  
final_dataframe = combined_df.reindex(columns = defined_columns)

final_dataframe.to_csv('{}_concatenated scraped NIST enzymes_01.csv'.format(datetime.date.today()))

## Column combination

The following code combined columns with akin data in the complete CSV. Suffixes were used to further specify data in columns that combined myriad data sources. The columns were finally renamed with generic conventions.

In [ ]:
import datetime 
import pandas 
import re

empty_cell = ['nan', 'NaN', 'none', 'not given', 'unknown', '0', '', None, ' ']

# dataframes definition
combined_dataframe = pandas.read_csv("2021-05-06_concatenated scraped NIST enzymes_01.csv")
df = combined_dataframe.astype(str)

combined_columns = []
regex_column_searches = ['(?i)(^K)', '(?i)(^K)', '(ë«|Î\´|δ)', '(I<sub>c)']
base_columns = ['K<sub>c</sub>\' ', 'K<sub>c</sub>\' ', 'δ<sub>r</sub>H(cal)/kJ mol<sup>-1</sup>)', 'I<sub>c</sub>(mol dm<sup>-3</sup>)']
print('\nColumns:\n', '='*len('Columns:'))
for this_column in df:
    print(this_column)
    for index, row in df.iterrows():
        for search in range(len(regex_column_searches)):
            # combine the like columns
            if re.search(regex_column_searches[search], this_column) and not re.search('(Km\')', this_column):
                if str(df.at[index, this_column]) not in empty_cell: 
                    if str(df.at[index, base_columns[search]]) in empty_cell:
                        df.at[index, base_columns[search]] = str(df.at[index, this_column])

                    if str(df.at[index, this_column]) not in empty_cell:
                        if str(df.at[index, base_columns[search]]) != str(df.at[index, this_column]):
                            df.at[index, base_columns[search]] = str(df.at[index, base_columns[search]]) + ' & ' + str(df.at[index, this_column])

                if this_column !=  base_columns[search]:
                    if this_column not in combined_columns:
                        combined_columns.append(this_column)

        if re.search('(?<=c\()(\w+\d?\+?)(?<!,)', this_column):
            if re.search('(?<=c\()(\w+\d?\+?)(?<!,)', this_column):
                solute = str(re.search('(?<=c\()(\w+\d?\+?)(?<!,)', this_column).group(1))
                
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)']) in empty_cell:
                    df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)'] = str(df.at[index, this_column]) + ' ' + solute

                if str(df.at[index, this_column]) not in empty_cell:
                    if str(df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)']) != (str(df.at[index, this_column]) or str(df.at[index, this_column]) + ' ' + solute):
                        df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)'] = str(df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)']) + ' & ' + str(df.at[index, this_column]) + ' ' + solute

            if this_column !=  'c(glycerol,mol dm<sup>-3</sup>)':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)

        if re.search('(?<=m\()(\w+\d?\+?)(?<!,)', this_column):
            if re.search('(?<=c\()(\w+\d?\+?)(?<!,)', this_column):
                solute = str(re.search('(?<=c\()(\w+\d?\+?)(?<!,)', this_column).group(1))
                
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)']) in empty_cell:
                    df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)'] = str(df.at[index, this_column]) + ' ' + solute

                if str(df.at[index, this_column]) not in empty_cell:
                    if str(df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)']) != (str(df.at[index, this_column]) or str(df.at[index, this_column]) + ' ' + solute):
                        df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)'] = str(df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)']) + ' & ' + str(df.at[index, this_column]) + ' ' + solute

            if this_column !=  'm(MgCl2,mol.kg<sup>-1</sup>)':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)

        buffer_columns = ['buffer(mol dm<sup>-3</sup>)', 'buffer and/or salt ', 'media ', 'buffer ']
        if this_column in buffer_columns:
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'Buffer:']) in empty_cell:
                    df.at[index, 'Buffer:'] = str(df.at[index, this_column])

                if str(df.at[index, 'Buffer:']) not in empty_cell:
                    if not re.search(re.escape(str(df.at[index, this_column])), str(df.at[index, 'Buffer:'])):
                        df.at[index, 'Buffer:'] = str(df.at[index, 'Buffer:']) + '  +  ' + str(df.at[index, this_column])
                        #print('Buffer: ERROR, index: ', index)             

            if this_column !=  'Buffer:':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)

        solution_columns = ['salt ', 'cosolvent ', 'added solute ', 'protein ', 'added solute ', 'percent(dimethyl sulfoxide) ', 'p(MPa)', 'pMg ']
        if this_column in solution_columns:
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'solvent ']) in empty_cell:
                    if str(this_column) == 'p(MPa)':
                        #print("yes: ", index)
                        df.at[index, 'solvent '] = str(df.at[index, this_column]) + ' megapascals'  

                    elif str(this_column) == 'pMg ':
                        #print('yes: ', index)
                        df.at[index, 'solvent '] = str(df.at[index, this_column]) + ' = -log[Mg+2]'   
                        
                    elif str(this_column) == 'percent(dimethyl sulfoxide) ':
                        df.at[index, 'solvent '] = str(df.at[index, this_column]) + ' % DMSO'   

                    else:
                        df.at[index, 'solvent '] = str(df.at[index, this_column])

                if str(df.at[index, 'solvent ']) not in empty_cell:
                    if not re.search(re.escape(str(df.at[index, this_column])), str(df.at[index, 'solvent '])):
                        df.at[index, 'solvent '] = str(df.at[index, 'solvent ']) + '  +  ' + str(df.at[index, this_column])

            if this_column !=  'solvent ':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)
            
# rename the base columns
df.rename(columns = {'c(glycerol,mol dm<sup>-3</sup>)':'solutes [mol / dm^3]', 
                     'I<sub>c</sub>(mol dm<sup>-3</sup>)':'Ionic strength [mol / dm^3]', 
                     'T(K)':'T [K]', 
                     'I<sub>m</sub>(mol.kg<sup>-1</sup>)':'Ionic strength [mol / kg]', 
                     'm(MgCl2,mol.kg<sup>-1</sup>)':'solutes [mol / kg]', 
                     'solvent ':'Experimental conditions', 
                     'K<sub>c</sub>\' ':'Keq', 
                     'δ<sub>r</sub>H(cal)/kJ mol<sup>-1</sup>)':'Enthalpy [kJ / mol]',
                     'Km\' ':'Km'},
          inplace = True)
          
print('\nDeleted columns:')
for column in combined_columns:
    print(column)
    del df[column]
    
#export total_df to csv 
df.to_csv("{}_vetted & reorganized NIST database_01.csv".format(datetime.date.today()))


# Comparing the new and old scraped data

The combined CSV of the NIST scrapped data is contrasted the complete set of IDs. The undescribed references in the recent scraping will be left unscraped, since these references in the NIST databse lack thermodynamic data and are therefore irrelevant, and possibly counterproductive, towards amassing a repository of thermodynamic data through our codes. 

In [71]:
import pandas
import bs4

final_df = pandas.read_csv('2021-05-06_concatenated scraped NIST enzymes_01.csv')
old_df = pandas.read_csv('2021-03-21_vetted + reorganized NIST_1.csv')
bs = BeautifulSoup(open('Enzyme Thermodynamic Database.html'), 'lxml')
table = bs.find("table", attrs = {'id': 'MainBody_gvSearch'})
body = table.find_all("tr")

described_ids = final_df['Reference ID:'].tolist()
old_described_ids = old_df['Reference ID:'].tolist()
reference_ids = []
described_reference_ids = []
undescribed_reference_ids = []
old_undescribed_reference_ids = []
for row in range(1, len(body)):
    id_value = body[row].find("a").text
    id_value = id_value.strip()
    reference_ids.append(id_value)   
    '''print(id_value)    '''
    
    if id_value not in described_ids:
        undescribed_reference_ids.append(id_value)

    if id_value not in old_described_ids:
        old_undescribed_reference_ids.append(id_value)
        
'''undescribed_reference_ids = set(reference_ids) - set(described_reference_ids)'''

for id in undescribed_reference_ids:
    print('Undescribed reference: ', id)
    
for id in old_undescribed_reference_ids:
    print('Old undescribed reference: ', id)
    
print('Total reference IDs: ', len(reference_ids))
print('Undescribed reference IDs: ', len(undescribed_reference_ids))
print('Old undescribed reference IDs: ', len(old_undescribed_reference_ids))

Undescribed reference:  54GIN/STU_1062
Undescribed reference:  54NOD/KUB_593
Undescribed reference:  56POD/MOR_1073
Undescribed reference:  57WOL/BAL_1173
Undescribed reference:  66WOO/DAV_1445
Undescribed reference:  67MOR/WHI_596
Undescribed reference:  68KOH/WAR_87
Undescribed reference:  69BLA_1166
Undescribed reference:  74CLA/BIR_1395
Undescribed reference:  74FLO/FLE_1063
Undescribed reference:  74LAN_648
Undescribed reference:  76GOL_548
Undescribed reference:  79COR/CRO_16
Undescribed reference:  79LAW/VEE_774
Undescribed reference:  79LAW/VEE_831
Undescribed reference:  84DEM_1065
Undescribed reference:  85DEM/BEH_1066
Undescribed reference:  86DAL/REN_1067
Undescribed reference:  89ROM/DEM_808
Undescribed reference:  89ROM/DEM_812
Undescribed reference:  91HOR/WAT_468
Undescribed reference:  91TEW/GOL2_655
Undescribed reference:  91TEW/GOL2_656
Undescribed reference:  93LAR/TEW_1053
Undescribed reference:  93LAR/TEW_1056
Old undescribed reference:  34MEY/LOH_1288
Old undescr